In [6]:
import sys
import os

# Add the parent directory to the path so the package is importable
sys.path.append(os.path.abspath(".."))

from llm_data_quality_assistant import pipeline
from llm_data_quality_assistant.corruptor import RowCorruptionTypes, CellCorruptionTypes
from llm_data_quality_assistant.enums import Models
import pandas as pd
from pprint import pprint
from dotenv import load_dotenv
import numpy as np

load_dotenv()


True

In [7]:
corrupt_dataset = pd.read_csv("../datasets/parker_datasets/allergen/allergen.csv")

gold_standard = pd.read_csv("../datasets/parker_datasets/allergen/gold_standard_alergene_pivoted.csv")

print(corrupt_dataset.head(2))
print(gold_standard.head(2))
print(type(gold_standard.get("code").iloc[0]))
print(type(corrupt_dataset.get("code").iloc[0]))

            code  nuts  almondnuts  brazil_nuts  macadamia_nuts  hazelnut  \
0  4104420005280     2           0            0               0         2   
1  4104420005280     2           0            0               0         0   

   pistachio  walnut  cashew  celery  ...  fish  gluten  lupin  milk  \
0          0       0       0       0  ...     0       2      2     1   
1          0       0       0       0  ...     0       2      2     1   

   molluscs  mustard  peanut  sesame  soy  sulfite  
0         0        0       0       0    1        0  
1         0        0       0       0    1        0  

[2 rows x 22 columns]
            code  nuts  almondnuts  brazil_nuts  macadamia_nuts  hazelnut  \
0  4104420006065     0           0            0               0         0   
1  4104420006065     0           0            0               0         0   

   pistachio  walnut  cashew  celery  ...  fish  gluten  lupin  milk  \
0          0       0       0       0  ...     0       2      0   

In [8]:
shortened_corrupt_df = corrupt_dataset[corrupt_dataset["code"].isin(gold_standard["code"])]

print("Shape shortened corrupt dataset:")
print(shortened_corrupt_df.shape)

print(shortened_corrupt_df)

shortened_gold_standard = gold_standard[gold_standard["code"].isin(shortened_corrupt_df["code"])]


print("Shape shortened gold standard dataset:")
print(shortened_gold_standard.shape)

print(shortened_gold_standard)


Shape shortened corrupt dataset:
(206, 22)
               code  nuts  almondnuts  brazil_nuts  macadamia_nuts  hazelnut  \
4     4104420006065     0           0            0               0         0   
5     4104420006065     0           0            0               0         0   
10    4104420007963     2           0            0               0         0   
11    4104420007963     2           0            0               0         2   
12    4104420007987     2           0            0               0         0   
...             ...   ...         ...          ...             ...       ...   
1131       42256199     1           0            0               0         0   
1146       42330660     0           0            0               0         0   
1147       42330660     0           0            0               0         0   
1154       42373186     1           0            0               0         0   
1155       42373186     1           0            0               0         0 

In [9]:
p = pipeline.Pipeline(shortened_gold_standard)

assert shortened_gold_standard.shape == shortened_corrupt_df.shape

# Find coordinates where entries differ
diff = shortened_gold_standard.values != shortened_corrupt_df.values
corrupted_coords = np.argwhere(diff)

print("Corrupted coordinates (row, col):")
print(corrupted_coords)

merged_df = p.clean_single_dataset(shortened_corrupt_df)

shortened_merged_df = merged_df[merged_df["code"].isin(gold_standard["code"])]

print("Shape merged dataset:")
print(merged_df.shape)

print("Shape shortened merged dataset:")
print(shortened_merged_df.shape)

results = p.evaluate_micro(shortened_merged_df, [corrupted_coords])
print(results)

same_values_only = (shortened_merged_df.values == shortened_corrupt_df.values).all()
print(same_values_only)


Corrupted coordinates (row, col):
[[  2   2]
 [  2   5]
 [  2   8]
 [  2  13]
 [  2  15]
 [  2  20]
 [  3   2]
 [  3   8]
 [  4   2]
 [  4   5]
 [  4   6]
 [  4   7]
 [  5   2]
 [  5   6]
 [  5   7]
 [  6   1]
 [  6   2]
 [  6  13]
 [  6  18]
 [  6  19]
 [  7   1]
 [  7   2]
 [  7  13]
 [  7  18]
 [  7  19]
 [ 10   2]
 [ 11   1]
 [ 11   2]
 [ 11  18]
 [ 15  13]
 [ 16   2]
 [ 16  13]
 [ 17   2]
 [ 18   1]
 [ 18   2]
 [ 18  18]
 [ 19   2]
 [ 21   9]
 [ 21  11]
 [ 21  15]
 [ 21  17]
 [ 21  20]
 [ 23   9]
 [ 30   2]
 [ 31   1]
 [ 31   2]
 [ 31  13]
 [ 31  18]
 [ 31  19]
 [ 31  20]
 [ 32   2]
 [ 32   8]
 [ 33   2]
 [ 33   8]
 [ 34   1]
 [ 34   2]
 [ 34   7]
 [ 34  18]
 [ 35   2]
 [ 35   7]
 [ 36   2]
 [ 37   1]
 [ 37   2]
 [ 37  18]
 [ 38   2]
 [ 39   1]
 [ 39   2]
 [ 39  18]
 [ 42   1]
 [ 42   2]
 [ 42   8]
 [ 42  18]
 [ 43   2]
 [ 43   8]
 [ 45   9]
 [ 45  17]
 [ 46   9]
 [ 46  17]
 [ 47   9]
 [ 47  17]
 [ 49   9]
 [ 49  17]
 [ 51   9]
 [ 51  17]
 [ 52  15]
 [ 54   2]
 [ 54   5]
 [ 55   1

JSONDecodeError: Expecting property name enclosed in double quotes: line 907 column 1 (char 15513)

In [ ]:
print("Shortened Merged DataFrame:")
print(shortened_merged_df)
print("Shortened Gold Standard DataFrame:")
print(shortened_gold_standard)

Shortened Merged DataFrame:
             code  nuts  almondnuts  brazil_nuts  macadamia_nuts  hazelnut  \
0   4104420006065     0           0            0               0         0   
1   4104420006065     0           0            0               0         0   
2   4104420007963     2           0            0               0         2   
3   4104420007963     2           0            0               0         2   
4   4104420007987     2           0            0               0         2   
5   4104420007987     2           0            0               0         2   
6   4104420010628     2           0            0               0         0   
7   4104420010628     2           0            0               0         0   
8   4104420014701     0           0            0               0         0   
9   4104420014701     0           0            0               0         0   
10  4104420015289     1           0            0               0         0   
11  4104420015289     1           0 